In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
import yfinance as yf
import zipfile
import matplotlib.pyplot as plt
import datetime
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow import keras
from matplotlib.dates import drange
from pandas import DataFrame
from pandas_datareader import data as pdr
from AlphaNet_Transformer.src.alphanet import AlphaNet, AlphaNetV3, load_model
from AlphaNet_Transformer.src.alphanet.data import TrainValData, TimeSeriesData
from AlphaNet_Transformer.src.alphanet.metrics import UpDownAccuracy

In [3]:
def strategy1(data):
    buy=[]
    sell=[]
    flag=-1
    profit = 0
    for i in range(len(data)):
        if data['Return'][i]>0:
            if flag!=1:
                buy.append(data['Close'][i])
                sell.append(np.nan)
                flag=1
            else:
                buy.append(np.nan)
                sell.append(np.nan)

        elif data['Return'][i]<0:
            if flag!=0:
                buy.append(np.nan)
                sell.append(data['Close'][i])
                flag=0
            else:
                buy.append(np.nan)
                sell.append(np.nan)
        else:
            buy.append(np.nan)
            sell.append(np.nan)
    return (buy, sell)
def strategy2(data):
            buy=[]
            sell=[]
            money = 0
            current = 0
            for i in range(len(data)):
                if data['Return'][i]>0:
                    buy.append(data['Close'][i])
                    sell.append(np.nan)
                    money = money - data['Close'][i]
                    current = current + 1
                elif data['Return'][i]<0:
                    buy.append(np.nan)
                    sell.append(data['Close'][i])
                    money = money + data['Close'][i] * current
                    current = 0
                else:
                    buy.append(np.nan)
                    sell.append(np.nan)
            return (buy, sell),money

In [ ]:
import warnings

warnings.filterwarnings('ignore')
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
profit = {}#5,10,20,30
for yield_duration in [5,10,20,30]:
    #20150131,20150430,20150731,20151031,20160131,20160430,20160731,20161031,20170131
    for l in [20150131,20150430,20150731,20151031,20160131,20160430,20160731,20161031,20170131]:
        print(l)
        date_begin = l
        label_name = str(yield_duration) + "-day_return"
        train_length = 800
        val_length = 150
        test_length = 150
        gap_length = yield_duration
        train_val_length = train_length + val_length + gap_length
        model_att_name = 'model_weights/' + str(yield_duration) + 'day/model_weights_att_' + str(date_begin) +'_' + str(train_length)
        model_lstm_name = 'model_weights/' + str(yield_duration) + 'day/model_weights_lstm_' + str(date_begin) + '_' + str(train_length)
        model_gru_name = 'model_weights/' + str(yield_duration) + 'day/model_weights_gru_' + str(date_begin) +'_' + str(train_length)

        model_att = AlphaNet(l2=0.001, dropout=0.1, recurrent_unit='Transformer')
        model_lstm = AlphaNetV3(l2=0.001, dropout=0, recurrent_unit='LSTM')
        model_gru = AlphaNetV3(l2=0.001, dropout=0, recurrent_unit='GRU')

        model_att.load_weights(model_att_name)
        model_lstm.load_weights(model_lstm_name)
        model_gru.load_weights(model_gru_name)

        path_data_100 = "./data_100_" + str(yield_duration) + "day.csv"
        data_all = pd.read_csv(path_data_100)
        df = data_all
        security_codes = ['OXY', 'BAC', 'RIG', 'XOM', 'F', 'SWN', 'CVX', 'ITUB', 'BBD', 'MRO', 'GOLD', 'NOK', 'WFC', 'UAL', 'C', 'DIS', 'CLF', 'SLB', 'SIRI', 'PBR', 'HAL', 'FCEL', 'AUY', 'ET', 'BP', 'FCX', 'KO', 'KGC', 'PLUG', 'CSX', 'TELL', 'GM', 'VTRS', 'CCJ', 'LYG', 'KOS', 'CVE', 'TSLA', 'BABA', 'DVN', 'JPM', 'ERIC', 'BTU', 'CDEV', 'PCG', 'PSXP', 'CS', 'ING', 'NUAN', 'MARA']
        profit_att_s1 = []
        profit_att_s2 = []
        profit_gru_s1 = []
        profit_gru_s2 = []

        for code in security_codes:
        #for code in ['FB']:
            #print(code)
            table_part = df.loc[df["code"] == code, :]
            stock_data_list = []
            stock_data_list.append(TimeSeriesData(dates=table_part["date"].values,                   # date column
                                                data=table_part.iloc[:, 3:].values,                # data columns
                                                labels=table_part[label_name].values)) # label column
            trainval_test_data = TrainValData(time_series_list=stock_data_list,
                                    train_length=train_val_length,   # 1200 trading days for training
                                    validate_length=test_length, # 150 trading days for validation
                                    history_length=30,   # each input contains 30 days of history
                                    sample_step=1,       # jump to days forward for each sampling
                                    train_val_gap=gap_length     # leave a 10-day gap between training and validation
                                    )
            train_val, test, dates_info = trainval_test_data.get(date_begin, order="by_date")
            # print(dates_info['validation']['start_date'])
            # print(dates_info['validation']['end_date'])
            pre_att = model_att.predict(test.batch(200))
            pre_lstm = model_lstm.predict(test.batch(200))
            pre_gru = model_gru.predict(test.batch(200))
            test_start_date = dates_info['validation']['start_date']
            test_end_date = dates_info['validation']['end_date']
            test = table_part[(table_part['date'] >= test_start_date) & (table_part['date'] <= test_end_date)]
            test.columns = ['code','Date','Return','Close','High','Low','Open','1','2','Volume','low/high']
            test = test.reset_index()
            test['Return'] = pre_att
            dt=pd.DataFrame()
            dt=test
            a1=strategy1(dt)
            dt['Buy Signal Price']=a1[0]
            dt['Sell Signal Price']=a1[1]
            temp = dt['Sell Signal Price'].sum() - dt['Buy Signal Price'].sum()
            profit_att_s1.append(temp)
            a1,money=strategy2(dt)
            profit_att_s2.append(money)
            test['Return'] = pre_gru
            dt=pd.DataFrame()
            dt=test
            a1=strategy1(dt)
            dt['Buy Signal Price']=a1[0]
            dt['Sell Signal Price']=a1[1]
            temp = dt['Sell Signal Price'].sum() - dt['Buy Signal Price'].sum()
            profit_gru_s1.append(temp)
            a1,money=strategy2(dt)
            profit_gru_s2.append(money)
        profit_att_s1 = np.array(profit_att_s1)
        profit[str(l) + '_' + str(yield_duration) + 'day_att_s1'] = np.average(profit_att_s1)
        profit_att_s2 = np.array(profit_att_s2)
        profit[str(l) + '_' + str(yield_duration) + 'day_att_s2'] = np.average(profit_att_s2)
        profit_gru_s1 = np.array(profit_gru_s1)
        profit[str(l) + '_' + str(yield_duration) + 'day_gru_s1'] = np.average(profit_gru_s1)
        profit_gru_s2 = np.array(profit_gru_s2)
        profit[str(l) + '_' + str(yield_duration) + 'day_gru_s2'] = np.average(profit_gru_s2)

In [ ]:
print(profit)